# Topic Modelling

## John Locke

In [2]:
with open("LockeComplete.txt", "r", encoding="utf-8") as file:
    Locke = file.read()

print(Locke)

an essay concerning human understanding by john locke contents an essay concerning human understanding epistle to the reader neither principles nor ideas are innate no innate speculative principles no innate practical principles other considerations concerning innate principles both speculative and practical of ideas of ideas in general and their original of simple ideas of simple ideas of sense idea of solidity of simple ideas of divers senses i of simple ideas of reflection ii of simple ideas of both sensation and reflection iii some further considerations concerning our simple ideas of sensation of perception of retention of discerning and other operations of the mind of complex ideas complex ideas of simple modesand first of the simple modes of the idea of space idea of duration and its simple modes ideas of duration and expansion considered together idea of number of infinity other simple modes of the modes of thinking of modes of pleasure and pain of power of mixed modes of our c

In [3]:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# NLTK
import nltk

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\Jason PC\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Jason PC\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\Jason PC\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['ibid'])

In [5]:
TokenList = nltk.word_tokenize(Locke)
LockeWords = nltk.Text(TokenList)
print(LockeWords[:50])

['an', 'essay', 'concerning', 'human', 'understanding', 'by', 'john', 'locke', 'contents', 'an', 'essay', 'concerning', 'human', 'understanding', 'epistle', 'to', 'the', 'reader', 'neither', 'principles', 'nor', 'ideas', 'are', 'innate', 'no', 'innate', 'speculative', 'principles', 'no', 'innate', 'practical', 'principles', 'other', 'considerations', 'concerning', 'innate', 'principles', 'both', 'speculative', 'and', 'practical', 'of', 'ideas', 'of', 'ideas', 'in', 'general', 'and', 'their', 'original']


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(LockeWords))

print(data_words[:5])

[['an'], ['essay'], ['concerning'], ['human'], ['understanding']]


In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\Jason PC\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['an']


In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:5])

C:\Users\Jason PC\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\Jason PC\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


[[], ['essay'], ['concern'], ['human'], ['understanding']]


In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:5])

[[], [(0, 1)], [(1, 1)], [(2, 1)], [(3, 1)]]


In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [12]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.127*"government" + 0.061*"war" + 0.035*"thing" + 0.029*"wherein" + '
  '0.025*"thus" + 0.023*"non" + 0.021*"tyranny" + 0.019*"scripture" + '
  '0.019*"atque" + 0.017*"obedience"'),
 (1,
  '0.095*"think" + 0.092*"resist" + 0.076*"public" + 0.049*"life" + '
  '0.044*"otherwise" + 0.041*"regimen" + 0.033*"lawful" + 0.029*"together" + '
  '0.024*"perfect" + 0.024*"barclay"'),
 (2,
  '0.148*"legislative" + 0.104*"good" + 0.072*"act" + 0.049*"never" + '
  '0.039*"liberty" + 0.034*"design" + 0.023*"action" + 0.023*"continue" + '
  '0.023*"attempt" + 0.021*"word"'),
 (3,
  '0.112*"case" + 0.089*"therefore" + 0.066*"use" + 0.060*"reason" + '
  '0.037*"seem" + 0.030*"governor" + 0.029*"earth" + 0.026*"populo" + '
  '0.022*"preserve" + 0.021*"soul"'),
 (4,
  '0.086*"consent" + 0.075*"would" + 0.065*"kind" + 0.057*"body" + '
  '0.042*"politic" + 0.041*"peace" + 0.032*"world" + 0.028*"forfeit" + '
  '0.025*"else" + 0.021*"mankind"'),
 (5,
  '0.101*"first" + 0.041*"much" + 0.036*"private" 

In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.921999019243307

Coherence Score:  0.7926782727232742


In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
0      8.697521        1       1 -0.291326  0.209080
17     5.968470        1       2 -0.033994 -0.018667
2      5.915741        1       3  0.080689 -0.077392
15     5.545609        1       4 -0.123326  0.026008
14     5.425210        1       5 -0.107920  0.008497
12     5.021152        1       6  0.068155 -0.004945
18     4.984582        1       7  0.053071 -0.011962
16     4.853610        1       8  0.014053 -0.048583
9      4.828853        1       9  0.017356 -0.066693
5      4.812148        1      10  0.015533 -0.080847
1      4.786688        1      11  0.045943 -0.074866
8      4.653383        1      12 -0.001279 -0.054803
11     4.533601        1      13  0.020737 -0.047041
13     4.506268        1      14  0.094727 -0.052960
4      4.440703        1      15  0.018318 -0.029439
19     4.385702        1      16  0.045885 -0.025212
3      4.234777        1      17  0.018636 -0.077695
7      4.224481        1      18  0.253096  0.367371
6      4.106234        1      19 -0.090936  0.015107
10     4.075269        1      20 -0.097419  0.045042, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
275   Default  2857.000000          man  2857.000000  30.0000  30.0000
858   Default  2117.000000          law  2117.000000  29.0000  29.0000
184   Default  1923.000000          may  1923.000000  28.0000  28.0000
239   Default  1743.000000         make  1743.000000  27.0000  27.0000
42    Default  1715.000000        power  1715.000000  26.0000  26.0000
1243  Default  1661.000000       people  1661.000000  25.0000  25.0000
3830  Default  1718.000000   government  1718.000000  24.0000  24.0000
4415  Default  1366.000000  legislative  1366.000000  23.0000  23.0000
871   Default  1156.000000        force  1156.000000  22.0000  22.0000
2120  Default  1129.000000         king  1129.000000  21.0000  21.0000
828   Default   952.000000      society   952.000000  20.0000  20.0000
321   Default   964.000000         good   964.000000  19.0000  19.0000
358   Default   843.000000          put   843.000000  18.0000  18.0000
877   Default   764.000000        state   764.000000  17.0000  17.0000
1066  Default   739.000000         case   739.000000  16.0000  16.0000
30    Default   756.000000        first   756.000000  15.0000  15.0000
829   Default   734.000000    authority   734.000000  14.0000  14.0000
3837  Default   724.000000       prince   724.000000  13.0000  13.0000
39    Default   707.000000        think   707.000000  12.0000  12.0000
222   Default   685.000000         give   685.000000  11.0000  11.0000
629   Default   717.000000          end   717.000000  10.0000  10.0000
2650  Default   684.000000       resist   684.000000   9.0000   9.0000
282   Default   686.000000         take   686.000000   8.0000   8.0000
6073  Default   831.000000          war   831.000000   7.0000   7.0000
146   Default   649.000000         must   649.000000   6.0000   6.0000
181   Default   640.000000         part   640.000000   5.0000   5.0000
1462  Default   668.000000          act   668.000000   4.0000   4.0000
377   Default   590.000000    therefore   590.000000   3.0000   3.0000
1384  Default   603.000000       person   603.000000   2.0000   2.0000
1188  Default   594.000000      consent   594.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
835   Topic20   322.406097       appeal   323.272003   3.1976  -2.9817
1461  Topic20   176.970901    community   177.836823   3.1954  -3.5815
128   Topic20   176.716599        order   177.582520   3.1953  -3.5829
482   Topic20   169.581268        agree   170.447189   3.1951  -3.6241
1343  Topic20   164.936157       remain   165.802078   3.1950  -3.6519
1318  Topic20   152.798798          due   153.664719   3.1946  -3.7284
719   Topic20   110.94

## George Berkeley

In [15]:
with open("BerkeleyComplete.txt", "r", encoding="utf-8") as file:
    Berkeley = file.read()

print(Berkeley)

a treatise concerning the principles of human knowledge wherein the chief causes of error and difficulty in the sciences with the grounds of scepticism atheism and irreligion are inquired into first printed in the year to which are added three dialogues between hylas and philonous in opposition to scepticks and atheists first printed in the year both written by george berkeley a fellow of trinitycollege dublin london printed for jacob tonson introduction philosophy being nothing else but the study of wisdom and truth it may with reason be expected that those who have spent most time and pains in it should enjoy a greater calm and serenity of mind a greater clearness and evidence of knowledge and be less disturbed with doubts and difficulties than other men yet so it is we see the illiterate bulk of mankind that walk the highroad of plain common sense and are governed by the dictates of nature for the most part easy and undisturbed to them nothing thats familiar appears unaccountable or

In [16]:
TokenList = nltk.word_tokenize(Berkeley)
BerkeleyWords = nltk.Text(TokenList)
print(BerkeleyWords[:50])

['a', 'treatise', 'concerning', 'the', 'principles', 'of', 'human', 'knowledge', 'wherein', 'the', 'chief', 'causes', 'of', 'error', 'and', 'difficulty', 'in', 'the', 'sciences', 'with', 'the', 'grounds', 'of', 'scepticism', 'atheism', 'and', 'irreligion', 'are', 'inquired', 'into', 'first', 'printed', 'in', 'the', 'year', 'to', 'which', 'are', 'added', 'three', 'dialogues', 'between', 'hylas', 'and', 'philonous', 'in', 'opposition', 'to', 'scepticks', 'and']


In [17]:
data_words = list(sent_to_words(BerkeleyWords))

print(data_words[:5])

[[], ['treatise'], ['concerning'], ['the'], ['principles']]


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\Jason PC\Anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[]


In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:5])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:5])

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

## David Hume

In [ ]:
with open("HumeComplete.txt", "r", encoding="utf-8") as file:
    Hume = file.read()

print(Hume)

In [ ]:
TokenList = nltk.word_tokenize(Hume)
HumeWords = nltk.Text(TokenList)
print(HumeWords[:50])

In [ ]:
data_words = list(sent_to_words(HumeWords))

print(data_words[:5])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:5])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:5])

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis